In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train_450 = pd.read_csv('ACT1_train_450.csv', dtype={"MOLECULE": object, "Act": float})

In [23]:
train_1 = pd.read_csv('ACT1_train_50_corr_red.csv', dtype={"MOLECULE": object, "Act": float})

In [24]:
def r_square(y, y_pred):
    """ r^2 value defined by the competition host, r^2 = 1 indicates 100% prediction accuracy
    """
    avx = np.mean(y)
    avy = np.mean(y_pred)
    sum1, sumx, sumy = 0, 0, 0
    for i in range(len(y)):
        sum1 += (y[i] - avx)*(y_pred[i] - avy)
        sumx += (y[i] - avx)*(y[i] - avx)
        sumy += (y_pred[i] - avy)*(y_pred[i] - avy)
#     print(len(y), sum1, sumx, sumy)
    return sum1*sum1/(sumx*sumy)

In [5]:
MAPE = []
def mean_ape(y_true, y_pred):
    return np.mean(np.abs((y_true-y_pred)/y_true))*100

In [25]:
def mean_absolute_error(y_true,y_pred):
    return np.mean(np.abs((y_true-y_pred)))

## Data set with 50 features

In [26]:
from sklearn.model_selection import train_test_split
import numpy as np
y = train_1['Act'].values
train_1 = train_1.drop(['Act', 'Unnamed: 0'], axis = 1)
x = train_1.values
X_train, X_test, Y_train, Y_test = train_test_split(x, y, train_size = 0.80, random_state = 0)



In [28]:
print(np.shape(Y_train), np.shape(X_train), np.shape(Y_test), np.shape(X_test))
Y_train = np.reshape(Y_train,(len(Y_train),1))
Y_test = np.reshape(Y_test,(len(Y_test),1))


(29792,) (29792, 49) (7449,) (7449, 49)


In [27]:
from sklearn.ensemble import RandomForestRegressor
#clean up output
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn.model_selection import cross_val_score
model = RandomForestRegressor(n_estimators=500, bootstrap = True, max_features = 'sqrt')




In [9]:
#custom R_2 score function
from sklearn.metrics import make_scorer
R_2 = make_scorer(r_square)
#clean up y
Y_train = np.ravel(Y_train)
print(Y_train)

[5.6186 4.3003 4.3003 ... 4.3003 5.0018 4.3003]


In [10]:
from sklearn.metrics import make_scorer
mean_ape = make_scorer(mean_ape)
#clean up y
Y_train = np.ravel(Y_train)
print(Y_train)

[5.6186 4.3003 4.3003 ... 4.3003 5.0018 4.3003]


In [29]:
mae = make_scorer(mean_absolute_error)

In [11]:
#R^2 score with CV
scores = cross_val_score(model,X_train, Y_train, cv = 7, scoring = R_2)
print(scores)
print('The average accuracy is:', scores.mean())

[0.62776075 0.60084598 0.62498934 0.59648879 0.59515798 0.59786118
 0.63639864]
The average accuracy is: 0.6113575212366309


In [17]:
#Mean absolute percentage error with 50 features selected with highest positive and negative correlation
scores = cross_val_score(model,X_train, Y_train, cv = 7, scoring = mean_ape)
print(scores)
print('The average accuracy is:', scores.mean())

[5.48462287 5.49954979 5.51523175 5.66788226 5.56143028 5.56866355
 5.54590512]
The average accuracy is: 5.549040803782197


In [30]:
scores = cross_val_score(model,X_train, Y_train, cv = 7, scoring = mae)
print(scores)
print('The average accuracy is:', scores.mean())

C:\Users\yanyi\Anaconda3\envs\data-x\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\yanyi\Anaconda3\envs\data-x\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\yanyi\Anaconda3\envs\data-x\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\yanyi\Anaconda3\envs\data-x\lib\site-packages\sklearn\model_selection\_validation.py:

[0.54346832 0.53024435 0.56185838 0.54334587 0.53402625 0.53243894
 0.55561385]
The average accuracy is: 0.5429994212757379


In [2]:
from xgboost import XGBRegressor

In [12]:
# fit model no training data
model = XGBRegressor(n_estimators=500)
model.fit(X_train, Y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [13]:
#xgboosting, n of estimator =500, MPE
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model,X_train, Y_train, cv = 7, scoring = mean_ape)
print(scores)
print('The average accuracy is:', scores.mean())

[6.54821207 6.59026131 6.67306159 6.69736206 6.6842866  6.69792027
 6.60722223]
The average accuracy is: 6.6426180192332875


## Data with 450 features

In [17]:
from sklearn.model_selection import train_test_split
import numpy as np
y = train_450['Act'].values
train_450 = train_450.drop(['Act', 'Unnamed: 0'], axis = 1)
x = train_450.values
X_train, X_test, Y_train, Y_test = train_test_split(x, y, train_size = 0.80, random_state = 0)

Y_train = np.reshape(Y_train,(len(Y_train),1))
Y_test = np.reshape(Y_test,(len(Y_test),1))

In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
model = RandomForestRegressor(n_estimators=500, bootstrap = True, max_features = 'sqrt')
scores = cross_val_score(model,X_train, Y_train, cv = 7, scoring = mean_ape)
print(scores)
print('The average accuracy is:', scores.mean())

C:\Users\yanyi\Anaconda3\envs\data-x\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\yanyi\Anaconda3\envs\data-x\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\yanyi\Anaconda3\envs\data-x\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\yanyi\Anaconda3\envs\data-x\lib\site-packages\sklearn\model_selection\_validation.py:

[10.86795915 10.65138472 11.13788837 10.76400107 10.69173433 10.69558481
 11.00108944]
The average accuracy is: 10.82994884167264


In [21]:
scores = cross_val_score(model,X_train, Y_train, cv = 7, scoring = mae)
print(scores)
print('The average accuracy is:', scores.mean())

C:\Users\yanyi\Anaconda3\envs\data-x\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\yanyi\Anaconda3\envs\data-x\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\yanyi\Anaconda3\envs\data-x\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\yanyi\Anaconda3\envs\data-x\lib\site-packages\sklearn\model_selection\_validation.py:

[0.53951921 0.53002489 0.55569263 0.53846576 0.52969399 0.53034763
 0.54976759]
The average accuracy is: 0.5390730983131071


In [22]:
scores = cross_val_score(model,X_train, Y_train, cv = 7, scoring = R_2)
print(scores)
print('The average accuracy is:', scores.mean())

C:\Users\yanyi\Anaconda3\envs\data-x\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\yanyi\Anaconda3\envs\data-x\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\yanyi\Anaconda3\envs\data-x\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\yanyi\Anaconda3\envs\data-x\lib\site-packages\sklearn\model_selection\_validation.py:

[0.67877234 0.64613523 0.66950718 0.64222986 0.64621683 0.65399973
 0.67868783]
The average accuracy is: 0.6593641416219309


In [21]:
# #R_2 value
# # Fit on training data
# model.fit(X_train, Y_train)
# acc = model.score(X_test, Y_test)
# predictions = model.predict(X_test)
# print(r_square(predictions,Y_test))